In [ ]:
# Dashboard to display survey results, based on bokeh

In [ ]:
# TODO: maybe only read in results from an evaluation

In [ ]:
import os
from pandas import read_csv
import pandas as pd
import random
import numpy as np
from bokeh.io import output_notebook, show, push_notebook
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, Div, Select, Slider, TextInput
from bokeh.plotting import figure as bokeh_figure
from os.path import dirname, join

from bokeh.palettes import Category20c
from bokeh.transform import factor_cmap
from bokeh.transform import cumsum
from bokeh.models import FactorRange

from masci_tools.vis.bokeh_plots import bokeh_scatter, bokeh_multi_scatter

In [ ]:
output_notebook()

In [ ]:
# To embed a bokeh sever hosted somewhere else in a webpage one can add (https://docs.bokeh.org/en/latest/docs/user_guide/embed.html#app-documents):
#from bokeh.embed import server_document
#script = server_document("https://demo.bokeh.org/sliders")
#script

In [ ]:
pwd = os.getcwd()

In [ ]:
datafilepath= join(pwd, 'dashboard/data/HMC_community_survey2021_merged_all_replies.csv')

In [ ]:
# read in data
surveydata = read_csv(datafilepath)

In [ ]:
surveydata.describe()

In [ ]:
# run evaluation

In [ ]:
x = [str(i) for i in range(7)]
xs = [x,x,x,x,x]
xlabels = [[f'1 pli bla blub i' for i in x], [f'2 pli bla blub i' for i in x], [f'3 pli bla blub i' for i in x], [f'4 pli bla blub i' for i in x], [f'5 pli bla blub i' for i in x]]
ys = [[random.randint(0, 1200) for i in x], [random.randint(0, 1200) for i in x], [random.randint(0, 1200) for i in x],[random.randint(0, 1200) for i in x], [random.randint(0, 1200) for i in x]]

In [ ]:
# create dashboard

In [ ]:
random

In [ ]:
random.randint
    
    




In [ ]:
print(random)

In [ ]:
# launch app

In [ ]:
# bokeh bar plot
def bokeh_barchart(df, x='value', y='counts', factors=None, figure=None, title='', width=0.9, xlabel='Answers', ylabel='Number of answers', palette=Category20c, fill_color='color'):
    """Plot an interactive bar chart with bokeh"""
    
    if isinstance(df, ColumnDataSource):
        xdata = df.data[x]
        if not 'color' in df.column_names:
            if not len(xdata) > 20:
                df.data['color'] = Category20c[len(xdata)] # ! if len(xdata)>20 this fails
    else:
        xdata = df[x]
        if not 'color' in df.columns:
            df['color'] = Category20c[len(xdata)] # ! if len(xdata)>20 this fails

    if figure is None:
        if factors is not None:
            fig = bokeh_figure(plot_height=600, plot_width=600,
               title=title,
               toolbar_location=None,
               x_range=FactorRange(factors=factors),
               tools='',#'hover',
               tooltips=[('Data', f'@{x}'), ('Count', f'@{y}')])
        else:
            fig = bokeh_figure(plot_height=600, plot_width=600,
               title=title,
               toolbar_location=None,
               tools='',#'hover',
               tooltips=[('Data', f'@{x}'), ('Count', f'@{y}')])
    
    #if factors is not None:
    #    fig.x_range=FactorRange(factors=factors)
    
    fig.vbar(x=x, top=y, width=width, source=df, line_color="white", fill_color=fill_color)#factor_cmap('x', palette=palette, factors=factors, start=1, end=2))
    fig.y_range.start = 0
    fig.x_range.range_padding = 0.1
    fig.xaxis.major_label_orientation = 1
    fig.yaxis.axis_label = ylabel
    fig.xaxis.axis_label = xlabel
    fig.xgrid.grid_line_color = None
    return fig

In [ ]:
# test
df_test = pd.DataFrame(data=dict(value=xs[0], counts=ys[0]))
fig = bokeh_barchart(df_test, factors=xs[0])
show(fig)

In [ ]:
fig.visible = False

In [ ]:
fig.update?

In [ ]:
fig.disabled = True

In [ ]:
fig.destroy?

In [ ]:

fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']
years = ['2015', '2016', '2017']
data = {'fruits' : fruits,
        '2015'   : [2, 1, 4, 3, 2, 4],
        '2016'   : [5, 3, 3, 2, 4, 6],
        '2017'   : [3, 2, 4, 4, 5, 3]}

# this creates [ ("Apples", "2015"), ("Apples", "2016"), ("Apples", "2017"), ("Pears", "2015), ... ]
x = [ (fruit, year) for fruit in fruits for year in years ]
counts = sum(zip(data['2015'], data['2016'], data['2017']), ()) # like an hstack
print(counts)

In [ ]:
df_test2 = ColumnDataSource(data=dict(value=x, counts=counts))

fig2 = bokeh_barchart(df_test2, factors=x, fill_color=factor_cmap('value', palette=palette, factors=years, start=1, end=2))
show(fig2)

In [ ]:
# test

counts2 = sum(zip(*ys), ())
factors = [(str(ques), ans) for ques in range(len(xs)) for ans in xs[0]] # factors have to be unique, and strings
df_test3 = ColumnDataSource(data=dict(value=factors, counts=counts2))
palette = Category20c[len(ys[0])]
fig = bokeh_barchart(df_test3, factors=factors, fill_color=factor_cmap('value', palette=palette, factors=xs[0], start=1, end=2))
show(fig)

In [ ]:
# bokeh piechart    
def bokeh_piechart(df, x='value', y='counts', figure=None, radius=0.6, title=''):
    """Draw an interactive piechart with bokeh"""
    
    from math import pi
    
    ydata = np.array(df[y])
    df['angle'] = ydata / sum(list(ydata)) * 2 * pi
    
    if not 'color' in df.columns:
        df['color'] = Category20c[len(ydata)] # ! if len(xdata)>20 this fails
    df['percent'] = ydata / sum(ydata)


    if figure is None:
        fig = bokeh_figure(plot_height=600, plot_width=600,
               title=title,
               toolbar_location=None,
               tools='hover',
               tooltips=[('Data', f'@{x}'),
                         ('Percent', '@percent{0.00%}'), 
                         ('Count', f'@{y}')])
    #fig.add_layout(Legend(), 'right')
    fig.wedge(x=0,
            y=1,
            radius=radius,
            start_angle=cumsum('angle', include_zero=True),
            end_angle=cumsum('angle'),
            line_color='white',
            fill_color='color',
            legend_field='xlabel',
            source=df)
    
    fig.axis.axis_label = None
    fig.axis.visible = False
    fig.grid.grid_line_color = None
    return fig

In [ ]:
# test
df_test = pd.DataFrame(data=dict(value=xs[0], counts=ys[0], xlabel=xs[0]))
fig = bokeh_piechart(df_test)
show(fig)

In [ ]:
# bokeh scatter plot
'''
def bokeh_scatter(x,
                  y=None,
                  *,
                  xlabel='x',
                  ylabel='y',
                  title='',
                  figure=None,
                  data=None,
                  saveas='scatter',
                  copy_data=False,
                  **kwargs):
    """Draw an interactive scatter with bokeh"""


    if isinstance(x, (dict, pd.DataFrame, ColumnDataSource)) or x is None:
        warnings.warn(
            'Passing the source as first argument is deprecated. Please pass in source by the keyword data'
            'and xdata and ydata as the first arguments', DeprecationWarning)
        data = x
        x = kwargs.pop('xdata', 'x')
        y = kwargs.pop('ydata', 'y')

    plot_data = process_data_arguments(data=data,
                                       x=x,
                                       y=y,
                                       copy_data=copy_data,
                                       single_plot=True,
                                       same_length=True,
                                       use_column_source=True)
    entry, source = plot_data.items(first=True)

    plot_params.set_defaults(default_type='function', name=entry.y)
    kwargs = plot_params.set_parameters(continue_on_error=True, **kwargs)

    p = plot_params.prepare_figure(title, xlabel, ylabel, figure=figure)

    plot_kwargs = plot_params.plot_kwargs(plot_type='scatter')
    res = p.scatter(x=entry.x, y=entry.y, source=source, **plot_kwargs, **kwargs)
    plot_params.add_tooltips(p, res, entry)

    if plot_params['level'] is not None:
        res.level = plot_params['level']

    plot_params.draw_straight_lines(p)
    plot_params.set_limits(p)
    plot_params.save_plot(p, saveas)

    return p
```

In [ ]:
fig = bokeh_multi_scatter([[x] for x in xs[0]], [[y] for y in ys[0]], marker_size=[int(x)+6 for x in xs[0]])

In [ ]:
questions = open(join(pwd, 'dashboard/data/questions.txt')).read().split()
QUESTION_MAP = {question : i for i, question in enumerate(questions)}


question_select = Select(title="Question", value="Question 1",
               options=questions)
filter_select = Select(title="Filter", value="All",
               options=open(join(pwd, 'dashboard/data/filters.txt')).read().split())
chart_select = Select(title="Filter", value="bar",
               options=['bar', 'pie'])
# Dublicate button


TOOLTIPS=[
    ("Title", "@title"),
    ("Answer", "@X"),
    ("Number of Answers", "@Y")
]


p = bokeh_figure(height=600, width=700, title="", toolbar_location='right', tooltips=TOOLTIPS, sizing_mode="scale_both")
source = ColumnDataSource(data=dict(x=[], y=[], color=[], title=[], year=[], revenue=[], alpha=[]))

In [ ]:

def select_data():
    """Select the data to display"""
    question = question_select.value
    index = QUESTION_MAP(question)
    selected = ColumnDataSource(data=dict(x=xs[index], y=ys[index], title=question))
    return selected


def select_chart():
    """Select how the data should be displayed"""
    charttype = chart_select.value.strip()
    return charttype

def update():
    """Update the charts"""
    df = select_data()
    charttype = select_chart()

In [ ]:
# develop interactive multi bar chart

In [ ]:
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show
from bokeh.transform import dodge
from pprint import pprint
from bokeh.palettes import Category20

In [ ]:
def update_bar(event):
    '''replot the bar chart, but leave out hidden items'''
    print('Python:Click')

In [ ]:


width = 0.2
fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']
years = ['2015', '2016', '2017', '2018', '2019', '2020']
years_visible = [False, True, True, True, True, True]
years_color = Category20[len(years)]#["#c9d9d3","#718dbf", "#e84d60", "#e84d60", "#e84d60", "#e84d60"]
position = []#for i in range(years_visible.count(True))]
step = width+0.05
nvisible = years_visible.count(True)
print(nvisible%2)
if nvisible%2 == 0:
    start = -step*nvisible/2 + step/2.0
elif nvisible==1:
    start = 0.0
else:
    start = nvisible//2 * -step
displayed_pos = [start+i*step for i in range(nvisible)]
#print(displayed_pos, start, step)
ind = 0
for visible in years_visible:
    if not visible:
        position.append(0.0)
    else:
        position.append(displayed_pos[ind])
        ind = ind+1

#print(position)
#position = [-0.25, 0.0, 0.25]
data = {'fruits' : fruits,
        '2015'   : [2, 1, 4, 3, 2, 4],
        '2016'   : [5, 3, 3, 2, 4, 6],
        '2017'   : [3, 2, 4, 4, 5, 3],
       '2018'   : [2, 1, 3, 6, 7, 2],
       '2019'   : [1, 1, 2, 2, 2, 1],
       '2020'   : [5, 5, 4, 4, 5, 5]}

# todo proper dynamic xrange, maybe use factor x_range instead always...
source = ColumnDataSource(data=data)
#x_axis = [ (fruit, year) for fruit in fruits for year in years ]

p = figure(x_range=fruits,#FactorRange(*x_axis), #fruits,#
           y_range=(0, 10), title="Fruit Counts by Year",
           height=600, toolbar_location=None, tools="")
for i, year in enumerate(years):
    p.vbar(x=dodge('fruits', position[i], range=p.x_range), top=year, source=source,
       width=width, color=years_color[i], legend_label=year, visible=years_visible[i])

p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None
p.legend.location = "top_left"
p.legend.orientation = "horizontal"
p.legend.click_policy="hide"

show(p)